In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

from src.data.load_data import load_raw_data
from src.data.data_profiler import check_data
import polars as pl

df_January = load_raw_data(year=2024, month=1)
df_February = load_raw_data(year=2024, month=2)
df_March = load_raw_data(year=2024, month=3)
df_April = load_raw_data(year=2024, month=4)
df_May = load_raw_data(year=2024,month=5)
df_June  = load_raw_data(year=2024, month=6)
df_July = load_raw_data(year=2024, month=7)
df_August = load_raw_data(year=2024, month=8)
df_September  = load_raw_data(year=2024, month=9)
df_October  = load_raw_data(year=2024, month=10)
df_November = load_raw_data(year=2024, month=11)
df_December = load_raw_data(year=2024, month=12)

In [2]:
dfs = [
    df_January, df_February, df_March, df_April, df_May, df_June,
    df_July, df_August, df_September, df_October, df_November, df_December
]

df_full = pl.concat(dfs)

In [3]:
check_data(df_full)



╔══════════════════════════════════════════════════════════╗
║               DATA PROFILING REPORT                      ║
╚══════════════════════════════════════════════════════════╝


  БАЗОВАЯ ИНФОРМАЦИЯ

Размер датасета:
  Строк: 5,860,568
  Колонок: 13
  Размер в памяти: 754.30 MB

Типы данных:
  Datetime(time_unit='us', time_zone=None): 2 колонок
  Float64: 4 колонок
  String: 7 колонок

Колонки:
   1. ride_id                        (String)
   2. rideable_type                  (String)
   3. started_at                     (Datetime(time_unit='us', time_zone=None))
   4. ended_at                       (Datetime(time_unit='us', time_zone=None))
   5. start_station_name             (String)
   6. start_station_id               (String)
   7. end_station_name               (String)
   8. end_station_id                 (String)
   9. start_lat                      (Float64)
  10. start_lng                      (Float64)
  11. end_lat                        (Float64)
  12. end_lng   

In [12]:
import polars as pl

def clean_rides_data(df: pl.DataFrame) -> pl.DataFrame:
    """
    Основная функция очистки данных о поездках
    """

    df_clean = df.clone()
    
    print("Начинаем очистку ...")
    print(f"Исходный размер: {df_clean.shape}")
    
    
    # Убедимся, что временные метки в правильном формате
    df_clean = df_clean.with_columns([
        pl.col("started_at").cast(pl.Datetime(time_unit="us")),
        pl.col("ended_at").cast(pl.Datetime(time_unit="us"))
    ])
    
    CHICAGO_LAT_BOUNDS = (41.6, 42.2)
    CHICAGO_LNG_BOUNDS = (-88.0, -87.5)
    
    coord_condition = (
        pl.col("start_lat").is_between(*CHICAGO_LAT_BOUNDS) &
        pl.col("start_lng").is_between(*CHICAGO_LNG_BOUNDS) &
        pl.col("end_lat").is_between(*CHICAGO_LAT_BOUNDS) &
        pl.col("end_lng").is_between(*CHICAGO_LNG_BOUNDS)
    )
    
    zero_coord_condition = (
        (pl.col("end_lat") != 0) & 
        (pl.col("end_lng") != 0)
    )
    
    df_clean = df_clean.filter(coord_condition & zero_coord_condition)
    print(f"После фильтрации координат: {df_clean.shape}")
    
    # длительность в минутах
    df_clean = df_clean.with_columns([
        ((pl.col("ended_at") - pl.col("started_at")).dt.total_seconds() / 60).alias("duration_minutes")
    ])
    
    time_condition = (
        (pl.col("duration_minutes") > 0) &  # Длительность положительная
        (pl.col("duration_minutes") <= 24 * 60) &  # Не более суток
        (pl.col("ended_at") >= pl.col("started_at"))  # Конец после начала
    )
    
    df_clean = df_clean.filter(time_condition & (pl.col("duration_minutes") >= 1))
    print(f"После фильтрации по времени: {df_clean.shape}")
    
    missing_analysis = df_clean.group_by("rideable_type").agg([
        pl.col("start_station_name").is_null().mean().alias("pct_missing_start_station"),
        pl.col("end_station_name").is_null().mean().alias("pct_missing_end_station"),
        pl.count().alias("count")
    ]).sort("count", descending=True)
    
    print("\nАнализ пропусков по типам самокатов:")
    print(missing_analysis)
    
    # Обычно electric bikes могут не иметь станций (бесдоковые)
    # Но для классических bikes станции должны быть
    
    #флаги для поездок без станций
    df_clean = df_clean.with_columns([
        pl.col("start_station_name").is_null().alias("missing_start_station"),
        pl.col("end_station_name").is_null().alias("missing_end_station")
    ])
    
    
    # Проверяем уникальность ride_id
    duplicate_ids = df_clean.group_by("ride_id").agg(pl.count().alias("count")).filter(pl.col("count") > 1)
    if duplicate_ids.height > 0:
        print(f"\nНайдено {duplicate_ids.height} дубликатов ride_id")
        # Оставляем первую запись для каждого ride_id
        df_clean = df_clean.unique(subset=["ride_id"], keep="first")

    valid_rideable_types = ["electric_bike", "classic_bike", "docked_bike"]
    valid_member_types = ["member", "casual"]
    
    df_clean = df_clean.filter(
        pl.col("rideable_type").is_in(valid_rideable_types) &
        pl.col("member_casual").is_in(valid_member_types)
    )
    
    # Поездки без конечной станции И без конечных координат
    potential_issues = df_clean.filter(
        pl.col("missing_end_station") & 
        ((pl.col("end_lat").is_null()) | (pl.col("end_lng").is_null()))
    )
    
    print(f"\nПотенциальные проблемные поездки (без конечной точки): {potential_issues.height}")
    
    # Поездки с очень большой длительностью
    long_rides = df_clean.filter(pl.col("duration_minutes") > 12 * 60)  # Более 12 часов
    print(f"Поездки длительностью > 12 часов: {long_rides.height}")

    # Добавляем день недели, час и месяц для анализа
    df_clean = df_clean.with_columns([
        pl.col("started_at").dt.weekday().alias("day_of_week"),
        pl.col("started_at").dt.hour().alias("hour_of_day"),
        pl.col("started_at").dt.month().alias("month"),
        pl.col("started_at").dt.date().alias("date")
    ])
    
    df_clean_version = df_clean.filter(
        ~pl.col("missing_start_station") & 
        ~pl.col("missing_end_station")
    )
    
    print(f"\nФинальные размеры:")
    print(f"- Полный очищенный набор: {df_clean.shape}")
    print(f"- Только поездки со станциями: {df_clean_version.shape}")
    
    # отчет об очистке
    report = {
        "initial_rows": df.height,
        "final_rows": df_clean.height,
        "rows_removed": df.height - df_clean.height,
        "pct_removed": round((1 - df_clean.height / df.height) * 100, 2),
        "missing_start_station_pct": round(df_clean["missing_start_station"].mean() * 100, 2),
        "missing_end_station_pct": round(df_clean["missing_end_station"].mean() * 100, 2),
        "avg_duration_minutes": round(df_clean["duration_minutes"].mean(), 2),
        "median_duration_minutes": round(df_clean["duration_minutes"].median(), 2)
    }
    
    print("\nОтчет об очистке:")
    for key, value in report.items():
        print(f"  {key}: {value}")
    
    return df_clean, df_clean_version, report

df_clean, df_clean_with_stations, report = clean_rides_data(df_full)

# Доп анализ аномалий
def analyze_anomalies(df: pl.DataFrame):
    """
    Анализ потенциальных аномалий и краж
    """
    print("\n" + "="*50)
    print("АНАЛИЗ АНОМАЛИЙ")
    print("="*50)
    
    # Поездки без начальной станции
    no_start_station = df.filter(pl.col("missing_start_station"))
    print(f"1. Поездки без начальной станции: {no_start_station.height}")
    print("   Распределение по типам самокатов:")
    print(no_start_station.group_by("rideable_type").agg(pl.count()).sort("count", descending=True))
    
    # Поездки без конечной станции
    no_end_station = df.filter(pl.col("missing_end_station"))
    print(f"\n2. Поездки без конечной станции: {no_end_station.height}")
    print("   Распределение по типам пользователей:")
    print(no_end_station.group_by("member_casual").agg(pl.count()).sort("count", descending=True))
    
    # Поездки с очень короткой/длинной длительностью
    print(f"\n3. Анализ длительности поездок:")
    print(f"   Менее 5 минут: {df.filter(pl.col('duration_minutes') < 5).height}")
    print(f"   Более 3 часов: {df.filter(pl.col('duration_minutes') > 180).height}")
    
    # Проверка круглосуточных поездок
    overnight_rides = df.filter(
        (pl.col("duration_minutes") > 60 * 6) &  # Более 6 часов
        (pl.col("started_at").dt.hour() >= 18) &  # Начало вечером
        (pl.col("ended_at").dt.hour() <= 8)       # Окончание утром
    )
    print(f"\n4. Ночные длительные поездки (>6 часов): {overnight_rides.height}")
    
    # Анализ по дням недели
    print(f"\n5. Распределение поездок по дням недели:")
    day_of_week_stats = df.group_by("day_of_week").agg([
        pl.count().alias("count"),
        pl.col("duration_minutes").median().alias("median_duration")
    ]).sort("day_of_week")
    print(day_of_week_stats)
    
    return {
        "no_start_station": no_start_station,
        "no_end_station": no_end_station,
        "overnight_rides": overnight_rides
    }

# Запускаем анализ
anomalies = analyze_anomalies(df_clean)

# Сохранение очищенных данных
def save_cleaned_data(df_full_clean: pl.DataFrame, df_with_stations: pl.DataFrame):
    """
    Сохранение очищенных данных
    """
    df_full_clean.write_parquet("data/processed/rides_2024_cleaned.parquet")
    df_with_stations.write_parquet("data/processed/rides_2024_with_stations.parquet")   
    df_with_stations.write_csv("data/processed/rides_2024_with_stations_sample.csv")
    
    print("\nДанные сохранены:")
    print("- rides_2024_cleaned.parquet (все очищенные данные)")
    print("- rides_2024_with_stations.parquet (только со станциями)")
    print("- rides_2024_with_stations_sample.csv (выборка для просмотра)")


save_cleaned_data(df_clean, df_clean_with_stations)

Начинаем очистку ...
Исходный размер: (5860568, 13)
После фильтрации координат: (5853236, 13)
После фильтрации по времени: (5721328, 14)

Анализ пропусков по типам самокатов:
shape: (3, 4)
┌──────────────────┬───────────────────────────┬─────────────────────────┬─────────┐
│ rideable_type    ┆ pct_missing_start_station ┆ pct_missing_end_station ┆ count   │
│ ---              ┆ ---                       ┆ ---                     ┆ ---     │
│ str              ┆ f64                       ┆ f64                     ┆ u32     │
╞══════════════════╪═══════════════════════════╪═════════════════════════╪═════════╡
│ electric_bike    ┆ 0.327891                  ┆ 0.329033                ┆ 2869052 │
│ classic_bike     ┆ 0.0                       ┆ 0.00001                 ┆ 2714692 │
│ electric_scooter ┆ 0.467278                  ┆ 0.470752                ┆ 137584  │
└──────────────────┴───────────────────────────┴─────────────────────────┴─────────┘


/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_6426/84470565.py:55: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("count")
/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_6426/84470565.py:72: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  duplicate_ids = df_clean.group_by("ride_id").agg(pl.count().alias("count")).filter(pl.col("count") > 1)



Найдено 168 дубликатов ride_id

Потенциальные проблемные поездки (без конечной точки): 0
Поездки длительностью > 12 часов: 2544

Финальные размеры:
- Полный очищенный набор: (5583576, 20)
- Только поездки со станциями: (4121445, 20)

Отчет об очистке:
  initial_rows: 5860568
  final_rows: 5583576
  rows_removed: 276992
  pct_removed: 4.73
  missing_start_station_pct: 16.85
  missing_end_station_pct: 16.91
  avg_duration_minutes: 15.85
  median_duration_minutes: 9.98

АНАЛИЗ АНОМАЛИЙ
1. Поездки без начальной станции: 940713
   Распределение по типам самокатов:
shape: (1, 2)
┌───────────────┬────────┐
│ rideable_type ┆ count  │
│ ---           ┆ ---    │
│ str           ┆ u32    │
╞═══════════════╪════════╡
│ electric_bike ┆ 940713 │
└───────────────┴────────┘

2. Поездки без конечной станции: 944002
   Распределение по типам пользователей:
shape: (2, 2)
┌───────────────┬────────┐
│ member_casual ┆ count  │
│ ---           ┆ ---    │
│ str           ┆ u32    │
╞═══════════════╪════════╡

/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_6426/84470565.py:148: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  print(no_start_station.group_by("rideable_type").agg(pl.count()).sort("count", descending=True))
/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_6426/84470565.py:154: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  print(no_end_station.group_by("member_casual").agg(pl.count()).sort("count", descending=True))
/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_6426/84470565.py:172: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("count"),


shape: (7, 3)
┌─────────────┬────────┬─────────────────┐
│ day_of_week ┆ count  ┆ median_duration │
│ ---         ┆ ---    ┆ ---             │
│ i8          ┆ u32    ┆ f64             │
╞═════════════╪════════╪═════════════════╡
│ 1           ┆ 750912 ┆ 9.3             │
│ 2           ┆ 768055 ┆ 9.15            │
│ 3           ┆ 839772 ┆ 9.4             │
│ 4           ┆ 796398 ┆ 9.333333        │
│ 5           ┆ 801937 ┆ 9.783333        │
│ 6           ┆ 879465 ┆ 11.85           │
│ 7           ┆ 747037 ┆ 11.666667       │
└─────────────┴────────┴─────────────────┘

Данные сохранены:
- rides_2024_cleaned.parquet (все очищенные данные)
- rides_2024_with_stations.parquet (только со станциями)
- rides_2024_with_stations_sample.csv (выборка для просмотра)


In [13]:

df_clean.write_parquet("cleaned_scooter_data_2024.parquet")

cleaning_stats = {
    "initial_rows": len(df_full),
    "cleaned_rows": len(df_clean),
    "removed_pct": (len(df_full) - len(df_clean)) / len(df_full) * 100,
    "remaining_columns": df_clean.shape[1]
}

print("Статистика очистки:")
for key, value in cleaning_stats.items():
    print(f"{key}: {value}")

Статистика очистки:
initial_rows: 5860568
cleaned_rows: 5583576
removed_pct: 4.72636781963796
remaining_columns: 20


In [14]:
def business_insights(df: pl.DataFrame):
    """
    Извлечение бизнес-инсайтов из очищенных данных
    """
    print("="*60)
    print("БИЗНЕС-АНАЛИЗ ДАННЫХ ПРОКАТА")
    print("="*60)
    
    # 1. Анализ сезонности
    print("\n1. СЕЗОННОСТЬ ПО МЕСЯЦАМ:")
    monthly_stats = df.with_columns(
        pl.col("started_at").dt.month().alias("month")
    ).group_by("month").agg([
        pl.count().alias("total_rides"),
        pl.mean("duration_minutes").alias("avg_duration"),
        (pl.col("member_casual") == "member").mean().alias("member_pct"),
        (pl.col("rideable_type") == "electric_bike").mean().alias("electric_bike_pct")
    ]).sort("month")
    
    print(monthly_stats)
    
    # 2. Анализ по часам (пиковые часы)
    print("\n2. ПИКОВЫЕ ЧАСЫ ИСПОЛЬЗОВАНИЯ:")
    hourly_usage = df.with_columns(
        pl.col("started_at").dt.hour().alias("hour")
    ).group_by("hour").agg([
        pl.count().alias("rides"),
        pl.mean("duration_minutes").alias("avg_duration"),
        (pl.col("member_casual") == "member").mean().alias("member_pct")
    ]).sort("hour")
    
    # Находим пиковые часы
    peak_hours = hourly_usage.sort("rides", descending=True).head(3)
    print("Топ-3 пиковых часа:")
    print(peak_hours)
    
    # 3. Анализ пользовательского поведения
    print("\n3. РАЗНИЦА МЕЖДУ MEMBER И CASUAL:")
    user_analysis = df.group_by("member_casual").agg([
        pl.count().alias("total_rides"),
        pl.mean("duration_minutes").alias("avg_duration"),
        pl.median("duration_minutes").alias("median_duration"),
        (pl.col("rideable_type") == "electric_bike").mean().alias("electric_bike_pct"),
        (pl.col("rideable_type") == "classic_bike").mean().alias("classic_bike_pct"),
        (pl.col("rideable_type") == "electric_scooter").mean().alias("scooter_pct"),
        # Пропуски станций по типам пользователей
        pl.col("start_station_name").is_null().mean().alias("missing_start_pct"),
        pl.col("end_station_name").is_null().mean().alias("missing_end_pct")
    ])
    
    print(user_analysis)
    
    # 4. Самые популярные станции
    print("\n4. ТОП-10 ПОПУЛЯРНЫХ СТАНЦИЙ (старт):")
    top_start_stations = df.filter(
        pl.col("start_station_name").is_not_null()
    ).group_by("start_station_name").agg([
        pl.count().alias("departures"),
        pl.mean("duration_minutes").alias("avg_duration"),
        (pl.col("member_casual") == "member").mean().alias("member_pct")
    ]).sort("departures", descending=True).head(10)
    
    print(top_start_stations)
    
    # 5. Среднее расстояние поездок
    print("\n5. АНАЛИЗ РАССТОЯНИЙ:")
    if "distance_km" in df.columns:
        distance_stats = df.filter(
            (pl.col("distance_km") > 0) & 
            (pl.col("distance_km") < 50)  # Фильтруем аномалии
        ).select([
            pl.mean("distance_km").alias("avg_distance_km"),
            pl.median("distance_km").alias("median_distance_km"),
            pl.max("distance_km").alias("max_distance_km"),
            pl.min("distance_km").alias("min_distance_km")
        ])
        
        print(distance_stats)
    
    # 6. Доходность поездок (оценка)
    print("\n6. ОЦЕНКА ДОХОДНОСТИ (предположительная):")
    # Предположим тариф: $0.25/мин для electric, $0.20/мин для classic
    revenue_estimate = df.with_columns(
        pl.when(pl.col("rideable_type") == "electric_bike")
          .then(pl.col("duration_minutes") * 0.25)
          .when(pl.col("rideable_type") == "electric_scooter")
          .then(pl.col("duration_minutes") * 0.25)
          .otherwise(pl.col("duration_minutes") * 0.20)
          .alias("estimated_revenue")
    ).select([
        pl.sum("estimated_revenue").alias("total_estimated_revenue"),
        pl.mean("estimated_revenue").alias("avg_revenue_per_ride"),
        pl.sum("duration_minutes").alias("total_minutes")
    ])
    
    print(revenue_estimate)

# Запускаем бизнес-анализ
business_insights(df_clean)

БИЗНЕС-АНАЛИЗ ДАННЫХ ПРОКАТА

1. СЕЗОННОСТЬ ПО МЕСЯЦАМ:
shape: (12, 5)
┌───────┬─────────────┬──────────────┬────────────┬───────────────────┐
│ month ┆ total_rides ┆ avg_duration ┆ member_pct ┆ electric_bike_pct │
│ ---   ┆ ---         ┆ ---          ┆ ---        ┆ ---               │
│ i8    ┆ u32         ┆ f64          ┆ f64        ┆ f64               │
╞═══════╪═════════════╪══════════════╪════════════╪═══════════════════╡
│ 1     ┆ 140207      ┆ 12.031686    ┆ 0.831349   ┆ 0.466689          │
│ 2     ┆ 218815      ┆ 13.287962    ┆ 0.78969    ┆ 0.368576          │
│ 3     ┆ 294684      ┆ 13.768273    ┆ 0.727447   ┆ 0.504826          │
│ 4     ┆ 404313      ┆ 15.212072    ┆ 0.684405   ┆ 0.542525          │
│ 5     ┆ 593093      ┆ 17.315549    ┆ 0.623801   ┆ 0.492459          │
│ …     ┆ …           ┆ …            ┆ …          ┆ …                 │
│ 8     ┆ 737724      ┆ 17.017299    ┆ 0.583191   ┆ 0.522865          │
│ 9     ┆ 662316      ┆ 15.978612    ┆ 0.618261   ┆ 0.526498     

/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_6426/1257393461.py:14: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("total_rides"),
/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_6426/1257393461.py:27: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("rides"),
/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_6426/1257393461.py:40: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("total_rides"),


shape: (2, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ member_ca ┆ total_rid ┆ avg_durat ┆ median_du ┆ … ┆ classic_b ┆ scooter_p ┆ missing_s ┆ missing_ │
│ sual      ┆ es        ┆ ion       ┆ ration    ┆   ┆ ike_pct   ┆ ct        ┆ tart_pct  ┆ end_pct  │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ str       ┆ u32       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ casual    ┆ 1998565   ┆ 21.918878 ┆ 12.616667 ┆ … ┆ 0.48285   ┆ 0.0       ┆ 0.165723  ┆ 0.177903 │
│ member    ┆ 3585011   ┆ 12.459295 ┆ 8.883333  ┆ … ┆ 0.488032  ┆ 0.0       ┆ 0.170015  ┆ 0.164142 │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

4. ТОП-10 ПОПУЛЯРНЫХ СТАНЦИЙ (старт):


/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_6426/1257393461.py:58: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("departures"),


shape: (10, 4)
┌─────────────────────────────────┬────────────┬──────────────┬────────────┐
│ start_station_name              ┆ departures ┆ avg_duration ┆ member_pct │
│ ---                             ┆ ---        ┆ ---          ┆ ---        │
│ str                             ┆ u32        ┆ f64          ┆ f64        │
╞═════════════════════════════════╪════════════╪══════════════╪════════════╡
│ Streeter Dr & Grand Ave         ┆ 64693      ┆ 32.188595    ┆ 0.227675   │
│ DuSable Lake Shore Dr & Monroe… ┆ 42789      ┆ 31.996827    ┆ 0.22667    │
│ DuSable Lake Shore Dr & North … ┆ 38699      ┆ 23.774354    ┆ 0.418254   │
│ Michigan Ave & Oak St           ┆ 38689      ┆ 30.922334    ┆ 0.367779   │
│ Kingsbury St & Kinzie St        ┆ 38318      ┆ 10.348066    ┆ 0.749256   │
│ Clark St & Elm St               ┆ 34530      ┆ 13.627182    ┆ 0.699131   │
│ Clinton St & Washington Blvd    ┆ 33373      ┆ 12.365581    ┆ 0.810296   │
│ Millennium Park                 ┆ 31793      ┆ 32.083828   